# 0. 드라이브 연결


In [7]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. .env파일 로드

In [8]:
import os
from dotenv import load_dotenv

load_dotenv("/content/drive/MyDrive/DILAB/YB/LLM_FineTuning/Qwen/.env")
API_KEY = os.getenv("ALIBABA_API_KEY")
print("API Key Loaded:", API_KEY[:5] + "..." if API_KEY else "없음")


API Key Loaded: sk-ec...


# 2. 파일 업로드 및 모델호출


In [13]:
import os
import requests
import json
from dotenv import load_dotenv

# 파일 경로
file_path = "/content/drive/MyDrive/DILAB/YB/LLM_FineTuning/Qwen/finetune_data.jsonl"

# .env 파일 로드
load_dotenv("/content/drive/MyDrive/DILAB/YB/LLM_FineTuning/Qwen/.env")

# API 키 가져오기
API_KEY = os.getenv("ALIBABA_API_KEY")

# API URL (수정됨)
API_URL = "https://dashscope-intl.aliyuncs.com/compatible-mode/v1/fine-tunes"

# 파일 업로드 URL
upload_url = "https://dashscope-intl.aliyuncs.com/compatible-mode/v1/files"

# 파일 업로드
with open(file_path, "rb") as file:
    upload_response = requests.post(
        upload_url,
        headers={"Authorization": f"Bearer {API_KEY}"},
        files={"file": file},
        data={"purpose": "batch"}
    )

# 업로드 응답 상태 코드 확인
print(f"업로드 응답 상태 코드: {upload_response.status_code}")

# 업로드 응답 본문 출력 (디버깅용)
print(f"업로드 응답 본문: {upload_response.text}")

# 응답 확인
if upload_response.status_code == 200:
    # 업로드된 파일 ID 확인
    file_id = upload_response.json().get("id")
    if file_id:
        print(f"업로드된 파일 ID: {file_id}")

        # 파인튜닝 요청
        fine_tune_payload = {
            "training_file": file_id, # 'file_id'가 아닌 'training_file'로 수정
            "model": "qwen-7b-chat",  # Qwen2.5-7B는 파인튜닝 모델 목록에 없을 수 있으므로 일반 모델로 변경
            "hyperparameters": {
                "n_epochs": 3,
                "learning_rate": 2e-5,
                "batch_size": 2
            },
            "suffix": "qwen-finetuned"
        }

        fine_tune_response = requests.post(
            API_URL,
            headers={"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"},
            data=json.dumps(fine_tune_payload)
        )

        # 파인튜닝 요청 결과 출력
        print(f"파인튜닝 요청 응답 상태 코드: {fine_tune_response.status_code}")
        print(f"파인튜닝 요청 응답 본문: {fine_tune_response.text}")

        try:
            fine_tune_response_json = fine_tune_response.json()
            print(fine_tune_response_json)
        except json.JSONDecodeError:
            print("파인튜닝 요청 응답을 JSON으로 파싱할 수 없습니다.")
            print(f"응답 본문: {fine_tune_response.text}")
    else:
        print("파일 업로드 후 파일 ID를 받지 못했습니다.")
else:
    print(f"파일 업로드 실패: {upload_response.status_code}")
    print(f"응답 메시지: {upload_response.text}")

업로드 응답 상태 코드: 200
업로드 응답 본문: {"id":"file-batch-84b21f8f6ad04f87a353f6b0","object":"file","bytes":4186921,"filename":"finetune_data.jsonl","purpose":"batch","status":"processed","created_at":1757557262}
업로드된 파일 ID: file-batch-84b21f8f6ad04f87a353f6b0
파인튜닝 요청 응답 상태 코드: 404
파인튜닝 요청 응답 본문: 


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [1]:
# @title
!pip install --upgrade requests urllib3
